In [1]:
from pulp import *
import numpy as np
import pandas as pd
model = LpProblem("SCheduling task", LpMinimize)

C:\Users\Csoni\AppData\Roaming\Python\Python39\site-packages\pulp\pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [2]:
#Read

df = pd.read_excel (r'Grace klinika.xlsx')
df_new1, df_new2 = pd.DataFrame(np.array(df.head(20))[:,1:]), pd.DataFrame(np.array(df.tail(len(df)-22))[1:,1:],np.array(df.tail(len(df)-22))[1:,0],np.array(df.tail(len(df)-22))[0,1:])
PG,GD=np.array(df_new1),np.array(df_new2)
P_index=np.array(df.head(20))[:,0]
G_index=np.array(df.tail(len(df)-22))[1:,0]
O_index=np.array(df.tail(len(df)-22))[0,1:]


for k1,e in enumerate(PG):
    for k2,e2 in enumerate(e):
        if np.isnan(e2):
            PG[k1,k2]=1

for k1,e in enumerate(GD):
    for k2,e2 in enumerate(e):
        if np.isnan(e2):
            GD[k1,k2]=1
            
print(P_index)
    

['Anna' 'Barna' 'Cecil' 'Domonkos' 'Eszter' 'Ferenc' 'Gergely' 'Huba'
 'Ilona' 'Janka' 'Kelemen' 'Lajos' 'Márta' 'Nóra' 'Olívia' 'Péter'
 'Róbert' 'Sámuel' 'Tibor' 'Ubul']


In [3]:
#Creat variable

#Doctors
indexes=[str([i,m,n]) for i in range(40) for m in range(3) for n in range(7)]
var_o = LpVariable.dicts("O", indexes, cat="Integer", lowBound=0,upBound=1)

#Patient
indexes=[str([i,m,n]) for i in range(40) for m in range(3) for n in range(20)]
var_p = LpVariable.dicts("P", indexes, cat="Integer", lowBound=0,upBound=1)

#Machines
indexes=[str([i,m,n]) for i in range(40) for m in range(3) for n in range(9)]
var_g = LpVariable.dicts("G", indexes, cat="Integer", lowBound=0,upBound=1)

#Value to optimise
var_opt=LpVariable.dicts("X",[0],lowBound=0,upBound=60)

In [4]:
#Boundaries

#Mindenki páciens legyen műtve
for n in range(20):
    model +=  lpSum(var_p[str([i,m,n])] for i in range(40) for m in range(3))==1

In [5]:
# Minden műtőben 1 páciens 2 doktor v SD
for i in range(40): 
    for m in range(3):
        model +=  lpSum(var_p[str([i,m,n])] for n in range(20))<=1
        model +=  2*lpSum(var_p[str([i,m,n])] for n in range(20))<=lpSum(var_o[str([i,m,n])] for n in range(1,7))+2*var_o[str([i,m,0])]

        #Minden páciens mellett van bent gép
        model += lpSum(var_g[str([i,m,g])] for g in range(9))>=lpSum(var_p[str([i,m,n])] for n in range(20))

In [6]:
# Minden gép 2 időegységet takarítva <-> 3 időegységenként legf 1 használat
for i in range(38):
    for n in range(9):
        model += lpSum(var_g[str([i+j,m,n])] for j in range(3) for m in range(3))<=1
        
# Doktor egyszerre egy helyen lehet
for n in range(7):
    for i in range(40):
        model += lpSum(var_o[str([i,m,n])] for m in range(3))<=1

for m in range(3):
    for i in range(40):
        model += lpSum(var_o[str([i,m,n])] for n in range(1,7))+2*var_o[str([i,m,0])]<=2
        
        
# Gépek szükségességi feltételei

for n in range(20):
    for m in range(3):
        #for i in range(40):
            for g in range(9):
                model += var_g[str([i,m,g])]<=  PG[n][g]
                model += var_g[str([i,m,g])] <= lpSum(var_o[str([i,m,o])]*GD[g][o] for o in range(7))
       
for n in range(20):
    for i in range(40):
        model += var_opt[0]>= (i+1)*lpSum(var_p[str([i,m,n])] for m in range(3))
model += var_opt[0]


In [7]:
#model.writeLP("HF4.lp")

In [8]:
listSolvers(onlyAvailable=True)

['CPLEX_CMD', 'CPLEX_PY', 'PULP_CBC_CMD']

In [9]:
solver = getSolver('PULP_CBC_CMD')
%timeit model.solve()

5.59 s ± 274 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
solver = getSolver('CPLEX_PY')
%timeit model.solve()

5.61 s ± 234 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [11]:
solver = getSolver('CPLEX_CMD')
%timeit model.solve()

5.7 s ± 389 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:
print("A szükséges minimális idő: "+str(model.objective.value()))

A szükséges minimális idő:7.0


In [13]:
#Print schedule

print("Páciensek beosztása: \n")
for n in range(20):
    for i in range(40):
        for m in range(3):
            if var_p[str([i,m,n])].value()==1:
                print(P_index[n]+" az "+str(i+1)+". időpontban menjen az "+str(m+1)+". műtőbe")
                
print("\n\n Az orvosok beosztása: \n")
for n in range(7):
    for i in range(40):
        for m in range(3):
            if var_o[str([i,m,n])].value()==1:
                print(O_index[n]+" az "+str(i+1)+". időpontban menjen az "+str(m+1)+". műtőbe")
                
print("\n\n A gépek beosztása: \n")
for n in range(9):
    for i in range(40):
        for m in range(3):
            if var_g[str([i,m,n])].value()==1:
                print(G_index[n]+" gép az "+str(i+1)+". időpontban legyen a "+str(m+1)+". műtőben")
                

                



Páciensek beosztása: 

Anna az 1. időpontban menjen az 2. műtőbe
Barna az 3. időpontban menjen az 1. műtőbe
Cecil az 3. időpontban menjen az 3. műtőbe
Domonkos az 7. időpontban menjen az 1. műtőbe
Eszter az 2. időpontban menjen az 3. műtőbe
Ferenc az 1. időpontban menjen az 1. műtőbe
Gergely az 5. időpontban menjen az 2. műtőbe
Huba az 5. időpontban menjen az 3. műtőbe
Ilona az 3. időpontban menjen az 2. műtőbe
Janka az 7. időpontban menjen az 2. műtőbe
Kelemen az 4. időpontban menjen az 1. műtőbe
Lajos az 4. időpontban menjen az 2. műtőbe
Márta az 5. időpontban menjen az 1. műtőbe
Nóra az 2. időpontban menjen az 2. műtőbe
Olívia az 6. időpontban menjen az 2. műtőbe
Péter az 7. időpontban menjen az 3. műtőbe
Róbert az 6. időpontban menjen az 1. műtőbe
Sámuel az 4. időpontban menjen az 3. műtőbe
Tibor az 2. időpontban menjen az 1. műtőbe
Ubul az 6. időpontban menjen az 3. műtőbe


 Az orvosok beosztása: 

SD az 1. időpontban menjen az 3. műtőbe
SD az 2. időpontban menjen az 2. műtőbe
SD